# Preprocessing of the data

Note: The final dataframes produced here can all be obtained by running the preprocessing.py file in the repository. 

In [2]:
# All imports for preprocessing

# import kagglehub
import numpy as np
import pandas as pd
import os
import re
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

# Download necessary NLTK data (run once if needed)
nltk.download('punkt')        # For tokenization
nltk.download('stopwords')    # For stopwords
nltk.download('wordnet')      # For lemmatization

# Initialize necessary NLP tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()  # For Lemmatization
stemmer = PorterStemmer()         # For Stemming (optional)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arthu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Import the data

In [3]:
movies = pd.read_csv('../data/MovieSummaries/movie.metadata.tsv', sep='\t', header=None)
characters = pd.read_csv('../data/MovieSummaries/character.metadata.tsv', sep = '\t', header = None)

# Importing txt files
name_clusters = pd.read_csv('../data/MovieSummaries/name.clusters.txt', sep = '\t', header = None)
summaries = pd.read_csv('../data/MovieSummaries/plot_summaries.txt', sep = '\t', header = None)
tv_tropes = pd.read_csv('../data/MovieSummaries/tvtropes.clusters.txt', sep = '\t', header = None)

We inspect the content of each dataframe

In [4]:
movies.sample(2)

,0,1,2,3,4,5,6,7,8
64894,29145570,/m/0dljlc4,The Yesterday Machine,NaN,NaN,NaN,{},{},"{""/m/06n90"": ""Science Fiction""}"
11501,23992427,/m/07kj9fj,The Unborn,1991-03-29,NaN,83.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama..."


In [5]:
characters.sample(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12
286594,262770,/m/01m_90,1997,NaN,NaN,NaN,NaN,NaN,David Toole,NaN,/m/0gcbv4l,NaN,/m/0gc0qy_
446214,5806110,/m/0f5zrv,1973-08-08,NaN,1942-05-14,F,NaN,NaN,Jennifer Billingsley,31.0,/m/0gcc7fc,NaN,/m/0gc2zbh


In [6]:
name_clusters.sample(2)

,0,1
130,Aldous Snow,/m/0648t1y
1829,Dylan Piper,/m/04djh5y


In [7]:
summaries.head(2)

,0,1
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...


In [8]:
tv_tropes.sample(2)

,0,1
39,brainless_beauty,"{""char"": ""Brace Channing"", ""movie"": ""My Favori..."
235,fastest_gun_in_the_west,"{""char"": ""Jim"", ""movie"": ""Blazing Saddles"", ""i..."


We observe that all the datafiles don't contain the column names. We name the columns according to the descriptions in the README. 

In [9]:
movies.columns = ['wikipedia_movie_id', 'freebase_movie_id', 'movie_name', 'movie_release_date', 'movie_box_office_revenue', 'movie_runtime', 'movie_languages', 'movie_countries', 'movie_genres']
name_clusters.columns = ['name', 'freebase_movie_id']
summaries.columns = ['wikipedia_movie_id', 'plot_summary']
characters.columns = ['wikipedia_movie_id', 'freebase_movie_id', 'movie_release_date', 'character_name', 'actor_birth', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 'actor_age', 'freebase_character_map', 'freebase_character_id', 'freebase_actor_id']
tv_tropes.columns = ['trope', 'details']

For the movies dataframe, we notice that for the languages, genres and countries we have dictionaries with freebase ids. We uniformize it into lists.

In [10]:
def extract_dict_to_list(entry):
    """
    Extracts values from a dictionary-like string and returns a list of values.
    """
    try:
        entry_dict = ast.literal_eval(entry)
        return list(entry_dict.values())
    except (ValueError, SyntaxError):
        return []

movies['movie_countries'] = movies['movie_countries'].apply(extract_dict_to_list)
movies['movie_genres'] = movies['movie_genres'].apply(extract_dict_to_list)
movies['movie_languages'] = movies['movie_languages'].apply(extract_dict_to_list)

In [11]:
movies.sample(2)

,wikipedia_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,movie_languages,movie_countries,movie_genres
49734,3729054,/m/09xpwr,Time to Leave,2006,NaN,77.0,[French Language],[France],"[LGBT, Family Drama, Drama, World cinema, Gay ..."
46611,17860039,/m/0479_qc,The Honor of the Family,1912-11-07,NaN,10.0,"[Silent film, English Language]",[United States of America],"[Short Film, Silent film, Drama, Black-and-white]"


For the tv_tropes dataframe, we notice that for the character, movie, freebase_movie_id and actor we have dictionaries which we uniformize into lists.

In [12]:
tv_tropes['details'] = tv_tropes['details'].apply(eval)
tv_tropes = tv_tropes.join(pd.json_normalize(tv_tropes['details'])).drop(columns=['details'])

In [13]:
tv_tropes.sample(2)

,trope,char,movie,id,actor
272,gentleman_thief,Neil McCauley,Heat,/m/0jz8sq,Robert De Niro
206,dumb_blonde,Michele Weinberger,Romy and Michele's High School Reunion,/m/0k2p5j,Lisa Kudrow


We rename the columns according to the descriptions in the README.

In [14]:
tv_tropes.columns = ['trope', 'character_name', 'movie_name', 'freebase_movie_id', 'actor_name']

We convert the dates into a unique format: datetime format. We also observe some of the dates only contain the year. We decide to keep only the year since the month and day won't be relevant for our analysis.

In [15]:
def extract_release_year(date_str):
    """
    Extracts the year from a date string.
    """
    try:
        return pd.to_datetime(date_str).year
    except (ValueError, TypeError):
        try:
            return int(date_str)
        except ValueError:
            return None

movies['movie_release_date'] = movies['movie_release_date'].apply(extract_release_year).astype(pd.Int64Dtype())


In [ ]:
# explain!!!
movies['movie_runtime'] = movies['movie_runtime'].fillna(movies['movie_runtime'].median())

### Preprocessing of the summaries

In [17]:
# Function for basic cleaning of text (Step 1)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Function for tokenization (Step 2)
def tokenize_text(text):
    return word_tokenize(text)

# Function to remove stopwords (Step 3)
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Function for lemmatization (Step 4) - Replace with stemming if needed
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

# Function for basic cleaning of text (Step 1)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Function for tokenization (Step 2)
def tokenize_text(text):
    return word_tokenize(text)

# Function to remove stopwords (Step 3)
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

# Function for lemmatization (Step 4)
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

# Apply text cleaning to 'plot_summary' in the 'summaries' DataFrame
summaries['clean_plot_summary'] = summaries['plot_summary'].apply(clean_text)

# Apply tokenization (Step 2)
summaries['tokenized_plot_summary'] = summaries['clean_plot_summary'].apply(tokenize_text)

# Apply stopword removal (Step 3)
summaries['filtered_tokens'] = summaries['tokenized_plot_summary'].apply(remove_stopwords)

# Apply lemmatization (Step 4)
summaries['lemmatized_tokens'] = summaries['filtered_tokens'].apply(lemmatize_tokens)

# Show the result
summaries[['plot_summary', 'clean_plot_summary', 'lemmatized_tokens']].head()

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\arthu/nltk_data'
    - 'c:\\Users\\arthu\\anaconda3\\envs\\ada\\nltk_data'
    - 'c:\\Users\\arthu\\anaconda3\\envs\\ada\\share\\nltk_data'
    - 'c:\\Users\\arthu\\anaconda3\\envs\\ada\\lib\\nltk_data'
    - 'C:\\Users\\arthu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# summaries.to_csv('../../data/preprocessed/summaries_preprocessed.csv')

In [18]:
''' Just for me here since my nltk isn't recognized for some reason'''
summaries = pd.read_csv('../data/processed/summaries_preprocessed.csv')

### Missing box office revenues

We notice that many movies don't have available box office revenues, which will be necessary for our analysis. 

In [19]:
print(f"Number of movies available: {movies.shape[0]}")
print(f"Number of movies with available box office: {movies['movie_box_office_revenue'].notna().sum()}")

Number of movies available: 81741
Number of movies with available box office: 8401


We download and inspect the kaggle IMDB 5000 Movie Dataset that will be used to fill missing values for box office revenue.

In [20]:
# # Download the dataset from kaggle and take read the movie_metadata.csv from the dataset
# ds_movies = kagglehub.dataset_download("carolzhangdc/imdb-5000-movie-dataset")
# imdb_movies = pd.read_csv(os.path.join(ds_movies, "movie_metadata.csv"))

# # Dataframe saved to csv for convenience
# local_path = "../data/raw/imdb_5000_movies.csv"
# os.makedirs(os.path.dirname(local_path), exist_ok=True)
# imdb_movies.to_csv(local_path, index=False)

imdb_movies = pd.read_csv('../data/raw/imdb_5000_movies.csv')

In [21]:
imdb_movies['movie_title'].sample(2)

3988      Menace II Society 
2193    Conan the Barbarian 
Name: movie_title, dtype: object

In [22]:
imdb_movies.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

We use the 'gross' column to fill in missing box_office_revenue values by merging on the movie names. 

In [23]:
imdb_movies['movie_title'] = imdb_movies['movie_title'].str.strip().str.replace(u'\xa0', '')

# Convert title_year to integer type, handling missing values as pd.NA
imdb_movies['title_year'] = imdb_movies['title_year'].fillna(0).astype(int).replace({0: pd.NA})
imdb_movies = imdb_movies[['movie_title', 'title_year', 'gross']]

# Convert 'gross' to numeric
imdb_movies['gross'] = pd.to_numeric(imdb_movies['gross'], errors='coerce')
imdb_movies = imdb_movies.dropna(subset=['gross'])

Since the kaggle IMDB 5000 Movie Dataset doesn't contain the wikipedia_movie_id, we merge the dataframes on the movie_name as well as the release_year. However, after inspection of the movies dataframe, we observe that that are duplicates on this subset that will cause issues with the merging. We observe exacly 27 cases of duplicates on this subset, all of them having at least one of the duplicates with a missing value in the movie_box_office_revenue column. <br>

We identify the duplicates on the ['movie_name', 'movie_release_date'] subset in the movies dataframe and check if they are also in the imdb_movies dataframe. We identify the pairs that have a missing value in the movie_box_office_revenue_column. 

In [24]:
# Group by on the ['movie_name', 'release_year'] subset in movies and count occurrences
duplicate_combinations = movies.groupby(['movie_name', 'movie_release_date']).size()
duplicates = duplicate_combinations[duplicate_combinations > 1].index

# Get the actual duplicates
duplicate_rows_in_movies = movies[movies[['movie_name', 'movie_release_date']].apply(tuple, axis=1).isin(duplicates)]

# Check if these duplicates are also in imdb_movies
matching_duplicates = duplicate_rows_in_movies[
    duplicate_rows_in_movies[['movie_name', 'movie_release_date']].apply(tuple, axis=1).isin(
        imdb_movies[['movie_title', 'title_year']].apply(tuple, axis=1)
    )
]

count_pairs_having_nan = 0
count_pairs_not_having_nan = 0
count_corresponding_imdb_movies = 0

# Find the cases that will lead to potential issues during merge
if not matching_duplicates.empty:
    for i, row in matching_duplicates.iterrows():
        sample_combination = row[['movie_name', 'movie_release_date']]

        sample_movies = movies[
            (movies['movie_name'] == sample_combination['movie_name']) &
            (movies['movie_release_date'] == sample_combination['movie_release_date'])
        ]

        if sample_movies['movie_box_office_revenue'].notnull().sum() == len(sample_movies):
            sample_imdb_movies = imdb_movies[
                (imdb_movies['movie_title'] == sample_combination['movie_name']) &
                (imdb_movies['title_year'] == sample_combination['movie_release_date'])
            ]
            count_pairs_not_having_nan += 1
        elif sample_movies['movie_box_office_revenue'].notnull().sum() != len(sample_movies):
            sample_imdb_movies = imdb_movies[
                (imdb_movies['movie_title'] == sample_combination['movie_name']) &
                (imdb_movies['title_year'] == sample_combination['movie_release_date'])
            ]
            count_pairs_having_nan += 1
            count_corresponding_imdb_movies += sample_imdb_movies.shape[0]
else:
    print("No duplicates found in movies that also match imdb_movies.")

In [25]:
print(f"Number of duplicates on subset: {len(matching_duplicates)}")
print(f"Duplicates with at least 1 missing value for box office revenue: {count_pairs_having_nan}")
print(f"Duplicates with no missing values for box office revenue: {count_pairs_not_having_nan}")
print(f"Count of corresponding imdb movies: {count_corresponding_imdb_movies}, meaning exactly 1 per duplicate")

Number of duplicates on subset: 27
Duplicates with at least 1 missing value for box office revenue: 27
Duplicates with no missing values for box office revenue: 0
Count of corresponding imdb movies: 27, meaning exactly 1 per duplicate


After inspection we decide to remove the movies part of these duplicate cases that do not have an available box office revenue. This is because the revenue available in imdb_movies dataframe has at most a 50% chance of being the revenue associated with the same movie based on the information given. Removing a total of 15 movies from the movies dataframe.

In [ ]:
# Remove appropriate movies from movies
movies_to_remove = matching_duplicates[matching_duplicates['movie_box_office_revenue'].isna()]
movies = movies[~movies['wikipedia_movie_id'].isin(movies_to_remove['wikipedia_movie_id'])]

# Print the number of movies removed
print(f"Number of movies removed: {len(movies_to_remove)}")

Number of movies removed: 15
Number of movies removed: 0


We now complete missing values of box office in the movies dataframe using the kaggle 5000 Movie Dataset for unambiguous cases.

In [28]:
# Convert 'movie_box_office_revenue' to numeric, handling missing values
movies['movie_box_office_revenue'] = pd.to_numeric(movies['movie_box_office_revenue'], errors='coerce')

# Merge on both 'movie_name' and 'movie_release_date' to differentiate versions
merged_movies = pd.merge(
    movies, imdb_movies,
    left_on=['movie_name', 'movie_release_date'],
    right_on=['movie_title', 'title_year'],
    how='left'
)

# Update 'movie_box_office_revenue' where it's NaN with 'gross' from IMDB
merged_movies['movie_box_office_revenue'] = merged_movies['movie_box_office_revenue'].fillna(merged_movies['gross'])
merged_movies.drop(columns=['movie_title', 'gross'], inplace=True)
merged_movies = merged_movies.dropna(subset=['movie_box_office_revenue'])

We can observe that a total of 538 missing box office revenues were filled in using the kaggle IMDB 5000 Movie Dataset.

In [30]:
print(f"Number of movies with available box office: {movies['movie_box_office_revenue'].notna().sum()}")
print(f"Number of movies available: {merged_movies.shape[0]}")
print(f"Count of values filled: {merged_movies.shape[0]-movies['movie_box_office_revenue'].notna().sum()}")

Number of movies with available box office: 8401
Number of movies available: 8939
Count of values filled: 538


### Choosing movies with summaries

We have a total of x films with box office revenue. In order to do our analysis, we also need the plot summaries of every one of those movies. We filter the dataframe obtained previously to have only movies with an available plot summary remaining.

In [31]:
common_index = merged_movies['wikipedia_movie_id'].isin(summaries['wikipedia_movie_id'])
filtered_movies_summaries_BO = merged_movies[common_index]
# We remove duplicates
filtered_movies_summaries_BO = filtered_movies_summaries_BO.drop_duplicates(subset='wikipedia_movie_id', keep='first')

In [32]:
filtered_movies_summaries_BO.shape[0]

7964

We are left with 7964 movies to which we add the plot summaries in the dataframe.

In [33]:
filtered_movies_summaries_BO = filtered_movies_summaries_BO.merge(
    summaries[['wikipedia_movie_id', 'plot_summary']],
    on='wikipedia_movie_id',
    how='left'
)

### Web scraping

In order to complete our dataset with additional features that can be useful for further analysis, a web scraping method that accesses IMDB movies web pages and extracts selected information on them was used. <br>The additional features extracted from this web scraping technique are: budget, opening_weekend, rating_score, number_of_ratings, producer.

Note: The web scraping takes several hours to run using the GPU provided on google colab. The code for the method can be found in the imdb_scraper.py and scrapping.ipynb files in the repository. <br>
<br>
We simply read the resulting csv as a dataframe here.

In [ ]:
''' Put correct directory here '''
#filtered_movies_summaries_BO_scraped = pd.read_csv('../data/processed/')

' Put correct directory here '

In [2]:
# Do preprocessing on missing values in the scraped df!!!